In [2]:
from bs4 import BeautifulSoup, Comment
import requests
import re
import copy
import webscraper

In [105]:
def find_annual_reports(cik):

    annual_reports = []
    header = {"User-Agent": "bzhang14@umd.edu"}
    base_url = "https://www.sec.gov/Archives/edgar/data"            # base url that every document starts with  
    directory_url = f"{base_url}/{cik}"                             # base Directoy List URL

    req = requests.get(directory_url, headers=header)   # sends get request to URL endpoint (html file)

    directory_webpage = BeautifulSoup(req.text, "html.parser")
    directory_table = directory_webpage.find("table")
    
    directory_links = directory_table.find_all("a")

    for a_tag in directory_links:
        subdirectory_url = f"{directory_url}/{a_tag.text}"
        req = requests.get(subdirectory_url, headers=header)
        subdirectory_webpage = BeautifulSoup(req.text, "html.parser")
        form_type, form_name = check_file_type(subdirectory_webpage, header, subdirectory_url)
        print(form_type)
        if form_type == None:
            print("error, form type is None")
        elif form_type == "Form 10-K":
            document_url = f"{subdirectory_url}/{form_name}"
            print(document_url)
            annual_reports.append(document_url)
    
    return annual_reports

def check_file_type(subdirectory_webpage: BeautifulSoup, header: dict, subdirectory_url: str) -> str:

    """
    Finds The File Type That Is Stored In A Sub-Directory Within The SEC EDGAR Directory Listing.

    Every subdirectory in the EDGAR Directory Listing has a index-headers.html file
    which is the first file in the subdirectory that has metadata about the filing itself!
    """
    subdirectory_table = subdirectory_webpage.find("table")
    subdirectory_link = list(subdirectory_table.find_all("a"))[1]   # gets the index.html file in the subdirectory
    file_name = subdirectory_link.text

    file_url = f"{subdirectory_url}/{file_name}"                # URL of the actual filed stored in Directory Listing

    # print(file_url)
    req = requests.get(file_url, headers=header)    
    index_file = BeautifulSoup(req.text, "html.parser")   

    form_type = index_file.find("div", {"id": "formName"}).find("strong").text    # check if the form is 10-K
    form_name = find_document_on_index_page(index_file)

    return form_type, form_name

def find_document_on_index_page(index_page: BeautifulSoup) -> str:
    file_table = index_page.find("table", {"class":"tableFile"})
    second_row = list(file_table.find_all("tr"))[1]
    document = list(second_row.find_all("td"))[2].find("a").text
    return document
    


In [2]:
annual_reports = webscraper.find_annual_reports("103379")

Form 8-K
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 10-Q
Form 8-K
Form 3
Form 3
Form 8-K
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 25-NSE
Form 8-K
Form 4
Form 4
Form 4
Form 4
Form 10-Q
Form 8-K
Form 8-K
Form 3
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 11-K
Form 8-K
Form ARS
Form DEFA14A
Form DEF 14A
Form 4
Form SD
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 10-K
https://www.sec.gov/Archives/edgar/data/103379/000010337923000006/vfc-20230401.htm
Form 8-K
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form SC 13G/A
Form CERT
Form 4
Form 3/A
Form 8-A12B
Form 8-K
Form 424B5
Form FWP
Form 424B3
Form 8-K
Form 4
Form 4
Form SC 13G/A
Form SC 13G/A
Form 4
Form SC 13G
Form SC 13G/A
Form SC 13G/A
Form 10-Q
Form 8-K
Form SC 13G
Form SC 13G/A
Form 8-K
Form 4
Form 4
Form 4
Form 4
Form 8-K
Form 10-Q
Form 8-K
Form 3
Form 3
Form 4
Form 4
Fo

In [108]:
print(len(annual_reports))

24


In [109]:
for report in annual_reports:
    print(report)

https://www.sec.gov/Archives/edgar/data/103379/000010337923000006/vfc-20230401.htm
https://www.sec.gov/Archives/edgar/data/103379/000010337922000006/vfc-20220402.htm
https://www.sec.gov/Archives/edgar/data/103379/000010337921000006/vfc-20210403.htm
https://www.sec.gov/Archives/edgar/data/103379/000010337920000006/vfc328202010-k.htm
https://www.sec.gov/Archives/edgar/data/103379/000010337919000006/vfc330201910-k.htm
https://www.sec.gov/Archives/edgar/data/103379/000010337918000003/vfc1230201710-k.htm
https://www.sec.gov/Archives/edgar/data/103379/000162828017002027/vfc1231201610-k.htm
https://www.sec.gov/Archives/edgar/data/103379/000119312516489832/d240426d10k.htm
https://www.sec.gov/Archives/edgar/data/103379/000119312515075174/d838659d10k.htm
https://www.sec.gov/Archives/edgar/data/103379/000119312514069474/d647537d10k.htm
https://www.sec.gov/Archives/edgar/data/103379/000119312513079974/d444924d10k.htm
https://www.sec.gov/Archives/edgar/data/103379/000119312512088711/d265930d10k.htm

In [72]:
url_list = []
f = open("../data/company data/VFC/forms.txt", "r")
flag = False
for line in f:
    if line == "file_type: 10-K\n":
        flag = True
    if flag and "url:" in line:
        flag = False
        url_list.append(line)
f.close()

print(url_list)

['url: https://www.sec.gov/Archives/edgar/data/103379/000010337923000006/0000103379-23-000006-index-headers.html\n', 'url: https://www.sec.gov/Archives/edgar/data/103379/000010337922000006/0000103379-22-000006-index-headers.html\n', 'url: https://www.sec.gov/Archives/edgar/data/103379/000010337921000006/0000103379-21-000006-index-headers.html\n', 'url: https://www.sec.gov/Archives/edgar/data/103379/000010337920000006/0000103379-20-000006-index-headers.html\n', 'url: https://www.sec.gov/Archives/edgar/data/103379/000010337919000006/0000103379-19-000006-index-headers.html\n', 'url: https://www.sec.gov/Archives/edgar/data/103379/000010337918000003/0000103379-18-000003-index-headers.html\n', 'url: https://www.sec.gov/Archives/edgar/data/103379/000162828017002027/0001628280-17-002027-index-headers.html\n', 'url: https://www.sec.gov/Archives/edgar/data/103379/000119312516489832/0001193125-16-489832-index-headers.html\n', 'url: https://www.sec.gov/Archives/edgar/data/103379/000119312515075174

In [78]:
year = 2023
for url in url_list:
    modified_string = url[5:-1]
    print(f"{year}:{modified_string}")
    year = year - 1

2023:https://www.sec.gov/Archives/edgar/data/103379/000010337923000006/0000103379-23-000006-index-headers.html
2022:https://www.sec.gov/Archives/edgar/data/103379/000010337922000006/0000103379-22-000006-index-headers.html
2021:https://www.sec.gov/Archives/edgar/data/103379/000010337921000006/0000103379-21-000006-index-headers.html
2020:https://www.sec.gov/Archives/edgar/data/103379/000010337920000006/0000103379-20-000006-index-headers.html
2019:https://www.sec.gov/Archives/edgar/data/103379/000010337919000006/0000103379-19-000006-index-headers.html
2018:https://www.sec.gov/Archives/edgar/data/103379/000010337918000003/0000103379-18-000003-index-headers.html
2017:https://www.sec.gov/Archives/edgar/data/103379/000162828017002027/0001628280-17-002027-index-headers.html
2016:https://www.sec.gov/Archives/edgar/data/103379/000119312516489832/0001193125-16-489832-index-headers.html
2015:https://www.sec.gov/Archives/edgar/data/103379/000119312515075174/0001193125-15-075174-index-headers.html


In [5]:
soup = BeautifulSoup(test.text, "html.parser")
table = soup.find("table")
table_entries = table.find_all(string=re.compile("^0000103379"))

print(len(list(table_entries)))

278


In [23]:
list_of_table_entries = list(table_entries)
base_url = "https://www.sec.gov/Archives/edgar/data/103379/"
base = "https://www.sec.gov"

In [42]:
# for index in range(0,50):
url = base_url + list_of_table_entries[0]

req = requests.get(url,headers={"User-Agent": "bzhang14@umd.edu"})
soup = BeautifulSoup(req.text, "html.parser")
table = soup.find("table")
td = table.find("td")
a = td.find("a")
link = a["href"]
url = base + link
print(url)
# print(table.prettify())
req = requests.get(url,headers={"User-Agent": "bzhang14@umd.edu"})
soup = BeautifulSoup(req.text, "html.parser")
#print(soup.prettify())
form_type = soup.find(text=lambda test:isinstance(test, Comment))
print(form_type)

s = str(form_type)
start="<TYPE>"
end = "<PUBLIC-DOCUMENT-COUNT>"
print(s[s.find(start) + len(start):s.rfind(end)])


https://www.sec.gov/Archives/edgar/data/103379/000010337923000018/0000103379-23-000018-index-headers.html

<SEC-HEADER>0000103379-23-000018.hdr.sgml : 20231103
<ACCEPTANCE-DATETIME>20231103154153
<ACCESSION-NUMBER>0000103379-23-000018
<TYPE>10-Q
<PUBLIC-DOCUMENT-COUNT>100
<PERIOD>20230930
<FILING-DATE>20231103
<DATE-OF-FILING-DATE-CHANGE>20231103
<FILER>
<COMPANY-DATA>
<CONFORMED-NAME>V F CORP
<CIK>0000103379
<ASSIGNED-SIC>2320
<IRS-NUMBER>231180120
<STATE-OF-INCORPORATION>PA
<FISCAL-YEAR-END>0330
</COMPANY-DATA>
<FILING-VALUES>
<FORM-TYPE>10-Q
<ACT>34
<FILE-NUMBER>001-05256
<FILM-NUMBER>231376123
</FILING-VALUES>
<BUSINESS-ADDRESS>
<STREET1>1551 WEWATTA STREET
<CITY>DENVER
<STATE>CO
<ZIP>80202
<PHONE>(720) 778-4000
</BUSINESS-ADDRESS>
<MAIL-ADDRESS>
<STREET1>1551 WEWATTA STREET
<CITY>DENVER
<STATE>CO
<ZIP>80202
</MAIL-ADDRESS>
<FORMER-COMPANY>
<FORMER-CONFORMED-NAME>VF CORPORATION
<DATE-CHANGED>19900621
</FORMER-COMPANY>
<FORMER-COMPANY>
<FORMER-CONFORMED-NAME>VANITY FAIR MILLS INC
<D

In [4]:
annual_reports = webscraper.find_annual_reports("1397187")

Form 4
Form 4
Form 4
Form 4
Form 144
Form 4
Form 144
Form 144
Form 144
Form 144
Form 4
Form 10-Q
Form 8-K
Form 4
Form 3
Form 8-K
Form 144
Form 4
Form 144
Form 144
Form 10-Q
Form 8-K
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form S-8 POS
Form S-8
Form 8-K
Form 4
Form 10-Q
Form 8-K
Form SD
Form SC 13D/A
Form DEFA14A
Form DEF 14A
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 10-K
https://www.sec.gov/Archives/edgar/data/1397187/000139718723000012/lulu-20230129.htm
Form 8-K
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form SC 13D/A
Form 4
Form 8-K
Form 5
Form 4
Form SC 13G/A
Form SC 13G/A
Form SC 13G/A
Form 4
Form SC 13G/A
Form SC 13D/A
Form 4
Form 8-K
Form 4
Form 4
Form 10-Q
Form 8-K
Form 4
Form 3
Form 8-K
Form 4
Form 10-Q
Form 8-K
Form S-3ASR
Form 4
Form 4
Form 8-K
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 4
Form 10-Q
Form 8-K
Form SD
Form DEFA14A
Form DEF 14A
Form 8-K
Form 4
Form 4
Form 4
Form 4

In [5]:
for report in annual_reports:
    print(report)

https://www.sec.gov/Archives/edgar/data/1397187/000139718723000012/lulu-20230129.htm
https://www.sec.gov/Archives/edgar/data/1397187/000139718722000014/lulu-20220130.htm
https://www.sec.gov/Archives/edgar/data/1397187/000139718721000009/lulu-20210131.htm
https://www.sec.gov/Archives/edgar/data/1397187/000139718720000012/lulu-20200202x10k.htm
https://www.sec.gov/Archives/edgar/data/1397187/000139718719000011/lulu-20190203x10k.htm
https://www.sec.gov/Archives/edgar/data/1397187/000139718718000013/lulu-20180128x10k.htm
https://www.sec.gov/Archives/edgar/data/1397187/000139718717000008/lulu-20170129x10k.htm
https://www.sec.gov/Archives/edgar/data/1397187/000139718716000089/lulu-20160131x10k.htm
https://www.sec.gov/Archives/edgar/data/1397187/000139718715000016/lulu-20150201x10k.htm
https://www.sec.gov/Archives/edgar/data/1397187/000139718714000021/lulu-20140202x10k.htm
https://www.sec.gov/Archives/edgar/data/1397187/000119312513118393/d475380d10k.htm
https://www.sec.gov/Archives/edgar/data